<a href="https://colab.research.google.com/github/priya170807/Time-seriesAnalysis/blob/main/WallStreetAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# basic libraries
import datetime
import numpy as np
import pandas as pd
# plotting libraries
import seaborn as sns
import matplotlib as plt
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools
%matplotlib inline
# ignore pandas warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv('/content/drive/MyDrive/august_wsj_pageviews.csv', skipinitialspace=True, infer_datetime_format=True)

In [3]:
df = data.copy()

In [4]:
df['pageview_time'] = pd.to_datetime(df['pageview_time'])

In [5]:
df['pageview_time'] = pd.to_datetime(df['pageview_time'].dt.strftime('%Y-%m-%d %H:%M:%S'))

In [6]:
df['publication_time'] = pd.to_datetime(df['publication_time'])

In [7]:
df['publication_time'] = pd.to_datetime(df['publication_time'].dt.strftime('%Y-%m-%d %H:%M:%S'))

In [8]:
df_olympics = df[df["article_type"]=="Olympics"]

In [9]:
df_olympics.shape

(1716, 15)

In [10]:
df_olympics["year"]=df["publication_time"].dt.year

In [11]:
# df_olympics_2021 = df_olympics[df_olympics["publication_time"].dt.year==2021]

In [12]:
# df_olympics.sort_values(by=["year"], inplace=True)

In [13]:
df_olympics_2021 = df_olympics[df_olympics["year"]==2021]

In [14]:
df_olympics_2021 = df_olympics_2021.reset_index(drop=True)

In [15]:
df_olympics_2021.shape

(1706, 16)

In [16]:
df_olympics_2021["section"].value_counts()

Life    1706
Name: section, dtype: int64

In [17]:
df_olympics_2021["subsection"].value_counts()

Sports    1706
Name: subsection, dtype: int64

In [18]:
# section and subsection don't carry any useful information so dropping
df_olympics_2021.drop(["section", "subsection", "article_type", "year"], axis=1, inplace=True)

In [19]:
df_olympics_2021.shape

(1706, 12)

In [20]:
# lets check the null values in the dataset now
df_olympics_2021.isnull().sum()

pageview_time          0
publication_time       0
visit_id               0
device_id              0
visit_page_num         0
subscriber_status      0
headline               0
author                11
device                 0
visit_ref_type       329
article_id             0
geo_country            0
dtype: int64

**Visit_ref_type:** lets see the most frequently occuring value for the visit_ref_type column and fill in the column with appropriate values instead of dropping the values.

**Author:** Since author cannot be determined lets use the value "unknown".

In [21]:
print(df_olympics_2021["visit_ref_type"].value_counts())
df_olympics_2021["visit_ref_type"]=df_olympics_2021["visit_ref_type"].fillna(value="Unknown")
df_olympics_2021["author"]=df_olympics_2021["author"].fillna(value="Unknown")

Social Network      592
Search Engines      475
external            154
Typed/Bookmarked    101
internal             55
Name: visit_ref_type, dtype: int64


In [22]:
df_olympics_2021["publication_date"]=df_olympics_2021["publication_time"].dt.date
df_olympics_2021["publication_hour"]=df_olympics_2021["publication_time"].dt.hour
df_olympics_2021["pageview_date"]=df_olympics_2021["pageview_time"].dt.date
df_olympics_2021["pageview_hour"]=df_olympics_2021["pageview_time"].dt.hour

In [23]:
df_olympics_2021.head()

,pageview_time,publication_time,visit_id,device_id,visit_page_num,subscriber_status,headline,author,device,visit_ref_type,article_id,geo_country,publication_date,publication_hour,pageview_date,pageview_hour
0,2021-07-31 02:33:04,2021-07-30 20:23:00,89046398,48613418,1,Non-subscriber,Mixed Relays in Swimming and Track Make Olympi...,Unknown,Mobile Web,Search Engines,WP-WSJ-0000039256,gbr,2021-07-30,20,2021-07-31,2
1,2021-07-16 16:49:26,2021-07-15 15:25:00,25313519,28923665,2,Non-subscriber,"When are the Tokyo Olympics? Schedule, Sports ...",Laine Higgins,Desktop,Unknown,WP-WSJ-0000035535,usa,2021-07-15,15,2021-07-16,16
2,2021-07-27 08:27:44,2021-07-27 11:47:00,87915003,57584918,1,Non-subscriber,Simone Biles Pulled Out of Gymnastics Team Com...,Louise Radnofsky|Georgi Kantchev|Andrew Beaton,Mobile Web,Social Network,WP-WSJ-0000038348,sgp,2021-07-27,11,2021-07-27,8
3,2021-07-30 02:31:51,2021-07-28 10:00:00,17208048,77131717,1,Non-subscriber,Fencers Refused to Fight. Then Came the Rule T...,Andrew Beaton,Mobile Web,Social Network,WP-WSJ-0000038685,usa,2021-07-28,10,2021-07-30,2
4,2021-07-23 16:49:07,2021-07-22 12:00:00,93348244,62514166,1,Non-subscriber,When the Tokyo Olympics Opening Ceremony Start...,Peter Landers,Mobile Web,Search Engines,WP-WSJ-0000036979,usa,2021-07-22,12,2021-07-23,16


In [24]:
df_olympics_2021["publication_date"]=pd.to_datetime(df_olympics_2021["publication_date"])
df_olympics_2021["pageview_date"]=pd.to_datetime(df_olympics_2021["pageview_date"])

**Bad data:**lets check whether the pageview_time is occuring only after publication_time when publication_date=pageview_date.

In [25]:
len(df_olympics_2021[df_olympics_2021["publication_time"]>df_olympics_2021["pageview_time"]])

237

In [26]:
len(df_olympics_2021[df_olympics_2021["publication_time"]<df_olympics_2021["pageview_time"]])

1469

**Filtering bad data:** Consider only good data where the pageview_time is occuring after the publication_time.

In [27]:
final_df = df_olympics_2021[df_olympics_2021["publication_time"]<df_olympics_2021["pageview_time"]]

In [28]:
final_df["pageview_time"] = final_df["pageview_time"].dt.floor('H')
final_df["publication_time"] = final_df["publication_time"].dt.floor('H')

In [29]:
final_df.shape

(1469, 16)

Lets find the count of number of visitors per day by grouping the pageview_time(date) and taking the count of number of visit_ids and plot the percentage of users per hour with respect to the total users per one particular day.

In [30]:
grouped_df_day = final_df.groupby("pageview_time")["visit_id"].agg(['count'])

In [31]:
grouped_df_day.columns = ["visit_count"]

In [32]:
grouped_df_day=grouped_df_day.reset_index()

In [33]:
grouped_df_day["date"] = grouped_df_day["pageview_time"].dt.date

In [34]:
grouped = grouped_df_day.groupby(["date"])["visit_count"].agg(['count','sum'])

In [35]:
grouped.columns = ["date_counts","total_visits_per_day"]

In [36]:
grouped = grouped.reset_index()

In [37]:
grouped_sum = grouped.loc[np.repeat(grouped.index.values,grouped["date_counts"])]

In [38]:
grouped_sum = grouped_sum.reset_index(drop=True)

In [39]:
percentage_df = pd.concat([grouped_df_day, grouped_sum], axis=1)

In [40]:
percentage_df["percentage_visits_per_hour"] = round((percentage_df["visit_count"]/percentage_df["total_visits_per_day"])*100, 2)

In [41]:
percentage_df.set_index("pageview_time",inplace=True )

In [42]:
# color=sns.color_palette()
def horizontal_bar_chart(percentage_df, color):
    trace = go.Bar(
        y=percentage_df.index,
        x=percentage_df.values,
        showlegend=False,
        name = "Percentage of Pageviews per hour",
        orientation = 'h',
        marker=dict(
            color='rgba(50, 171, 96, 0.6)',
            line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1)
        ),
    )
    return trace

In [43]:
# trace1 = horizontal_bar_chart(percentage_df["visit_count"].head(20), 'rgba(50, 171, 96, 0.6)')
trace2 = horizontal_bar_chart(percentage_df["percentage_visits_per_hour"].head(20), 'rgba(50, 171, 96, 0.6)')

In [44]:
fig = tools.make_subplots(rows=1,cols=1, vertical_spacing=0.5,
                          subplot_titles=[""])

In [45]:
# labeling the bar with percentages.
annotations=[]
for xpos, ypos in zip(percentage_df.index[:20], percentage_df["percentage_visits_per_hour"].head(20)):
  annotations.append(dict(
                          xref='x',
                          yref='y',
                          y=xpos,
                          x=ypos+2,
                          text=str(ypos) +'%',
                          font=dict(family='Arial', size=12,
                                    color='rgb(50, 171, 96)'),
                          showarrow=False
                          )
                          )


In [46]:
# fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)

fig.update_layout(
    title="percentage_of_pageviews_per_hour_per_day",
    height=600, 
    width=600, 
    annotations=annotations, 
    legend=dict(x=0.029, y=1.038, font_size=10),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',)
py.iplot(fig, filename='pageviews-plots')

In [48]:
df2 = percentage_df.copy()

In [49]:
df2 = df2.reset_index()
df2['hour'] = df2["pageview_time"].dt.hour

lets calculate how many people visited in the first hour of all days, second hour of all days...so on

In [50]:
# create new empty dataframe new_df so that we get all values of hours for each day to plot the graph.
date_list = pd.date_range('2021-07-01', periods=744, freq='H')

In [51]:
new_df=pd.DataFrame({"Timestamp":date_list, "hourly_count":0})

In [52]:
# use left join and merge the dataframes new_df on left with df2 on right on the pageview_time as the key.
merged_df = pd.merge(new_df, df2, how="left", left_on="Timestamp", right_on="pageview_time")

In [53]:
merged_df.isnull().sum()

Timestamp                       0
hourly_count                    0
pageview_time                 363
visit_count                   363
date                          363
date                          363
date_counts                   363
total_visits_per_day          363
percentage_visits_per_hour    363
hour                          363
dtype: int64

In [54]:
merged = merged_df[["Timestamp", "visit_count", "total_visits_per_day"]]
merged["Date"] = merged["Timestamp"].dt.date
merged["Date"] = pd.to_datetime(merged["Date"])
merged["Hour"] = merged["Timestamp"].dt.hour
merged.drop("total_visits_per_day", axis=1, inplace=True)
merged["visit_count"]=merged["visit_count"].fillna(value=0)

In [55]:
visits_per_hour_alldays = merged.groupby(["Hour"])["visit_count"].agg(["sum"])
visits_per_hour_alldays.columns=["sum_per_hour_per_day"]

In [56]:
visits_per_hour_alldays["percentage"]=round((visits_per_hour_alldays["sum_per_hour_per_day"]/(visits_per_hour_alldays["sum_per_hour_per_day"].sum())*100),2)

In [57]:
dataf = visits_per_hour_alldays.copy()

In [58]:
hour_list=["Midnight", "1:00AM","2:00AM","3:00AM","4:00AM","5:00AM","6:00AM","7:00AM","8:00AM","9:00AM","10:00AM",
           "11:00AM","Noon","1:00PM","2:00PM","3:00PM","4:00PM","5:00PM","6:00PM","7:00PM","8:00PM","9:00PM","10:00PM",
           "11:00PM"]
dataf["hour_list"]=hour_list
dataf = dataf.reset_index()
dataf.set_index("hour_list", inplace=True)


In [59]:
dataf.sort_values("percentage", ascending=False, inplace=True)

In [60]:
# dataf = dataf.drop(["Hour","sum_per_hour_per_day"], axis=1)

In [61]:
# 'rgb(128, 0, 128)'
# color=sns.color_palette()
def horizontal_bar_chart(df, color):
    print(df.index)
    print(df.values)
    trace = go.Bar(
        y=df.index,
        x=df.values,
        showlegend=False,
        name = "Percentage of Cummulative Pageviews per hour(all days)",
        orientation = 'h',
        marker=dict(
            color='rgb(128, 0, 128)',
            line=dict(
            color='rgb(128, 0, 128)',
            width=1)
        ),
    )
    return trace

In [62]:
trace1 = horizontal_bar_chart(dataf["percentage"], 'rgb(128, 0, 128)')
# trace2 = horizontal_bar_chart(percentage_df["percentage_visits_per_hour"].head(20), 'rgba(50, 171, 96, 0.6)')

Index(['2:00PM', '1:00PM', 'Noon', '4:00PM', '9:00PM', '8:00AM', '11:00AM',
       '3:00PM', '5:00PM', '9:00AM', '8:00PM', '10:00PM', '7:00PM', '6:00PM',
       '10:00AM', 'Midnight', '11:00PM', '7:00AM', '1:00AM', '6:00AM',
       '5:00AM', '2:00AM', '4:00AM', '3:00AM'],
      dtype='object', name='hour_list')
[6.94 6.33 6.13 6.13 5.51 5.45 5.45 5.31 5.17 5.17 5.11 4.7  4.63 4.56
 4.22 3.74 3.68 2.59 2.38 2.11 1.5  1.23 1.09 0.88]


In [63]:
fig = tools.make_subplots(rows=1,cols=1, vertical_spacing=0.5,
                          subplot_titles=[""])

In [64]:
# labeling the bar with percentages.
annotations=[]
for xpos, ypos in zip(dataf.index, dataf["percentage"]):
  annotations.append(dict(
                          xref='x',
                          yref='y',
                          y=xpos,
                          x=ypos+1,
                          text=str(ypos) +'%',
                          font=dict(family='Arial', size=12,
                                    color='rgb(128, 0, 128)'),
                          showarrow=False
                          )
                          )


In [65]:
# fig.append_trace(trace1, 1, 1)
fig.append_trace(trace1, 1, 1)

fig.update_layout(
    title="Percentage of Cummulative Pageviews per hour(all days)",
    height=600, 
    width=600, 
    annotations=annotations, 
    yaxis=dict(autorange="reversed"),
    legend=dict(x=0.029, y=1.038, font_size=10),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',)
py.iplot(fig, filename='pageviews-plots')

Now lets check the number of visits to the article by checking whether the visit is caused by subscriber or a non-subscriber.

In [89]:
# lets take the final_df as our base dataframe
subscriber_df = final_df.groupby(["subscriber_status", "pageview_date"])["visit_id"].agg("count")
subscriber_df = subscriber_df.unstack(level=0)
subscriber_df.fillna(value=0, inplace=True)

In [110]:
# 'rgb(128, 0, 128)'
# color=sns.color_palette()
def scatter_chart(df, color, name):
    # print(df.index)
    # print(df.values)
    trace = go.Scatter(
        x=df.index,
        y=df.values,
        name=name,
        # orientation = 'h',
        marker=dict(
            color=color,
            line=dict(
            color=color,
            width=1)
        ),
    )
    return trace

In [111]:
trace1 = scatter_chart(subscriber_df["Non-subscriber"], 'rgb(128, 0, 128)', "Non-subscribers")
trace2 = scatter_chart(subscriber_df["Subscriber"], 'rgba(50, 171, 96, 0.6)', "Subscribers")

In [112]:
fig = tools.make_subplots(rows=1,cols=1, vertical_spacing=0.5)

In [113]:
fig.add_trace(trace1)
fig.add_trace(trace2)

fig.update_layout(
    title="Count of Pageviews per subscriber status",
    height=600, 
    width=600, 
    # annotations=annotations, 
    xaxis=dict(tickangle=-45),
    legend=dict(x=0.029, y=1.038, font_size=10),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',)
py.iplot(fig, filename='pageviews-plots')

Now let us check the visit_count of the visitors by the reference field that is by Social network, Search engine etc

In [135]:
# lets take the final_df again for ou analysis purpose.
# final_df.isnull().sum()
reference_type_df = final_df.groupby(["device", "visit_ref_type"])["visit_id"].agg("count")
reference_type_df = reference_type_df.unstack(level=0)

In [136]:
reference_type_df

device,Desktop,Mobile Web
visit_ref_type,,
Search Engines,69,358
Social Network,30,481
Typed/Bookmarked,19,65
Unknown,114,138
external,65,84
internal,27,19


In [143]:
reference_type_df.columns.tolist()

['Desktop', 'Mobile Web']

In [181]:
# color=sns.color_palette()
fig = tools.make_subplots(rows=1,cols=2, shared_yaxes=True, vertical_spacing=0.04)
def horizontal_bar_chart(df, color, item):
    print(df.index)
    sorted_df=df.sort_values(ascending=False)
    print(sorted_df.values)
    trace = go.Bar(
        y=sorted_df.index,
        x=sorted_df.values,
        # showlegend=False,
        name = item,
        orientation = 'h',
       marker=dict(
            color=color,
            line=dict(
            color=color,
            width=0.5)
        ),
    )
    return trace

color_list=['rgb(128, 0, 128)', 'rgba(50, 171, 96, 0.6)']
for i, value in enumerate(zip(reference_type_df.columns.tolist(), color_list)):
  trace = horizontal_bar_chart(reference_type_df[value[0]], value[1], value[0])
  fig.add_trace(trace, 1, i+1)

fig.update_layout(
    title="Count of Pageviews per subscriber status",
    height=650, 
    width=650, 
    # annotations=annotations, 
    # yaxis=dict(autorange="reversed"),
    legend=dict(x=1, y=1, font_size=10),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',)
fig['layout']['xaxis1'].update(title='', range=[0, 500], dtick=50, autorange=False)
fig['layout']['xaxis2'].update(title='', range=[0, 500], dtick=50, autorange=False)

py.iplot(fig, filename='pageviews-plots')

Index(['Search Engines', 'Social Network', 'Typed/Bookmarked', 'Unknown',
       'external', 'internal'],
      dtype='object', name='visit_ref_type')
[114  69  65  30  27  19]
Index(['Search Engines', 'Social Network', 'Typed/Bookmarked', 'Unknown',
       'external', 'internal'],
      dtype='object', name='visit_ref_type')
[481 358 138  84  65  19]
